<a href="https://colab.research.google.com/github/khadelya/yolov5_ATLAS/blob/master/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_%E2%84%961_%D0%90%D0%B4%D0%B5%D0%BB%D1%8F_%D0%A5%D1%83%D1%81%D0%BD%D1%83%D0%BB%D0%BB%D0%B8%D0%BD%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Используя цикл for:
  Скачать 100 вакансий ( регион-Москва, название вакансии - Аналитик), поля name и salary в два отдельных списка (list)

Используя цикл while:
  Скачать 100 вакансий (регион-Санкт-Петербург, название вакансии - Аналитик), поля name и salary   в два отдельных списка (list)
  
Используя любой из видов цикла - создать список названий вакансий, больших по зп в Санкт-Петербурге чем, в Москве. Для оценки зарплаты брать нижнюю границу (from), очищенную от налога (-13%), и пересчитанную с учетом курса валют в рубли. 

Вывести длину списка, и самую высокоплачиваемую вакансию, с указанием названия и зп, из него, используя команду print


In [ ]:
import requests

In [ ]:
def is_vacancy_valid(vacancy):
  if vacancy['salary'] is None:
    return False
  salary = vacancy['salary']
  if salary['from'] is None:
    return False
  return True

In [ ]:
msc_names=[]
msc_salaries=[]
url="https://api.hh.ru/vacancies?text=Аналитик&area=1&search_field=name&per_page=100"
items=requests.get(url).json()['items']
for item in items:
  if is_vacancy_valid(item):
    msc_names+=[item['name']]
    msc_salaries+=[item['salary']] 

In [ ]:
spb_names=[]
spb_salaries=[]
url="https://api.hh.ru/vacancies?text=Аналитик&area=2&search_field=name&per_page=100"
items=requests.get(url).json()['items']
n=0
while n<len(items):
  item = items[n]
  if is_vacancy_valid(item):
    spb_names+=[item['name']]
    spb_salaries+=[item['salary']] 
  n+=1

In [ ]:
def get_salary_clean(salary):
  if salary['currency']=='RUR':
    coef=1
  else:
    url="https://v6.exchangerate-api.com/v6/cc1614fb1c963ca32ced8635/latest/{}".format(salary['currency'])
    exchange_rate=requests.get(url).json()
    coef=exchange_rate['conversion_rates']['RUB']
  if salary['gross']==True:
    net=0.87
  else:
    net=1
  salary_clean=round(salary['from']*coef*net,2)
  return salary_clean

In [ ]:
spb_over_msc={}
for i in range(len(spb_names)):
  val=''
  for j in range(len(msc_names)):
    if spb_names[i]==msc_names[j]:
      spb_salary=get_salary_clean(spb_salaries[i])
      msc_salary=get_salary_clean(msc_salaries[j])
      if spb_salary>msc_salary:
        val+='True'
      else:
        val+='False'
  if not ('False' in val or val==''):
    spb_over_msc[spb_names[i]]=spb_salary

In [ ]:
if spb_over_msc:
  spb_over_msc_sorted=sorted(spb_over_msc.items(), key=lambda item: item[1])
  lenght=len(spb_over_msc_sorted)
  highest_vacancy=list(spb_over_msc_sorted)[-1]
  print(f"Длина списка: {lenght}")
  print(f"Название самой высокооплачиваемой вакансии: {highest_vacancy[0]}, зарплата: {highest_vacancy[1]} руб.")
else:
  print("В скачанных вакансиях нет таких, чтобы при одинаковом названии зарплата в Санкт-Петербурге была больше чем, в Москве.")

Длина списка: 2
Название самой высокооплачиваемой вакансии: Финансовый аналитик, зарплата: 100000 руб.
